In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten

import cv2

import pandas as pd
import numpy as np
import matplotlib.image as mpimg

from utils import DataFrame
import keras
keras.__version__

Using TensorFlow backend.


'2.2.2'

In [2]:
# Model Parameters 
BATCH_SIZE = 128
NUMBER_OF_EPOCHS = 10
VALIDATION_SIZE = 0.15

#https://keras.io/optimizers/
OPTIMIZER_TYPE = "adam"
LOSS_TYPE = "mse"

INPUT_H, INPUT_W = 75, 200
INPUT_SHAPE = (INPUT_H, INPUT_W, 3)
# Crop parameters
YSTART = 110
YSTOP = 230

print("=========")
red_frame = DataFrame("./road-images-2/new-red.csv")
print("=========")
green_frame = DataFrame("./road-images-2/new-green.csv")


csv filepath:  ./road-images-2/new-red.csv
number of images:  7282
sample data: 
                                                NAME  STEER
0  ./road-images-2/red-flat/eagle_2018_09_14_20_2...    0.0
1  ./road-images-2/red-flat/eagle_2018_09_14_20_2...    0.0
2  ./road-images-2/red-flat/eagle_2018_09_14_20_2...    0.0
3  ./road-images-2/red-flat/eagle_2018_09_14_20_2...    0.0
4  ./road-images-2/red-flat/eagle_2018_09_14_20_2...    0.0


csv filepath:  ./road-images-2/new-green.csv
number of images:  6487
sample data: 
                                                NAME  STEER
0  ./road-images-2/green-flat/eagle_2018_09_14_20...    0.0
1  ./road-images-2/green-flat/eagle_2018_09_14_20...    0.0
2  ./road-images-2/green-flat/eagle_2018_09_14_20...    0.0
3  ./road-images-2/green-flat/eagle_2018_09_14_20...    0.0
4  ./road-images-2/green-flat/eagle_2018_09_14_20...    0.0



In [3]:
def random_flip(image, steer):
    if np.random.random() > 0.5:
        return np.fliplr(image), -steer
    else:
        return image, steer

In [4]:
def get_processed_data(x, data):
    
    i = data.index[x]
    steer =  data['STEER'][i]
    img = mpimg.imread(data['NAME'][i])
    img = img[YSTART:YSTOP, :, :]
    img = cv2.resize(img, (INPUT_W, INPUT_H), cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    
    return random_flip(img, steer)

In [5]:
def generate_samples(data, batch_size):

    while True:

        SIZE = len(data)
        data.sample(frac=1) #shuffle

        for start in range(0, SIZE, batch_size):
            images, steers = [], []

            for i in range(start, start + batch_size):
                if i < SIZE:
                    image, steer = get_processed_data(i, data)
                    steers.append(steer)
                    images.append(image)

            yield (np.array(images), np.array(steers))         

In [6]:
def build_modified_nvidia_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, (5, 5), activation='elu', strides=(2, 2)))
    model.add(Conv2D(36, (5, 5), activation='elu', strides=(2, 2)))
    model.add(Conv2D(48, (5, 5), activation='elu', strides=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='elu'))
    model.add(Conv2D(64, (3, 3), activation='elu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.compile(optimizer=OPTIMIZER_TYPE, loss=LOSS_TYPE)
    return model

In [7]:
def train_model(model, data, modelh5_name, modeljson_name):
    
    print('summary of model:')
    model.summary()
    

    TRAINING_DATA, VALIDATION_DATA = train_test_split(data, test_size=VALIDATION_SIZE)
    TOTAL_TRAIN_DATA = len(TRAINING_DATA)
    TOTAL_VALID_DATA = len(VALIDATION_DATA)
    
    print("Total training data:", TOTAL_TRAIN_DATA)
    print("Total validation data:", TOTAL_VALID_DATA)

    print('Training model...')
    train_generator = generate_samples(TRAINING_DATA, batch_size=BATCH_SIZE)
    validation_generator = generate_samples(VALIDATION_DATA, batch_size=BATCH_SIZE)

    model.fit_generator(
        train_generator,
        validation_data=validation_generator,         
        steps_per_epoch=TOTAL_TRAIN_DATA // BATCH_SIZE,
        validation_steps=TOTAL_VALID_DATA // BATCH_SIZE,
        epochs=NUMBER_OF_EPOCHS,
        #verbose=1,
    )
    
    print('...Model trained.')
    print('Saving model...')
    model.save(modelh5_name)

    with open(modeljson_name, "w") as json_file:
        json_file.write(model.to_json())

    print("...Model Saved at:", modelh5_name, "and", modeljson_name, ".")
    
    return model

In [8]:
red_model = build_modified_nvidia_model()

In [ ]:
red_model = train_model(red_model, red_frame.data, "./models/red2.h5", "./models/red2.json")

summary of model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 75, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 22, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 20, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 18, 64)         36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 18, 64)         0  

In [ ]:
green_model = build_modified_nvidia_model()
green_model = train_model(green_model, green_frame.data, "./models/green2.h5", "./models/green2.json")

summary of model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 75, 200, 3)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 36, 98, 24)        1824      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 47, 36)        21636     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 22, 48)         43248     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 20, 64)         27712     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 2, 18, 64)         36928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 18, 64)         0  